# Import libraries

In [1]:
import os
import torch

from huggingface_hub import login
from datasets import load_dataset
from unsloth import FastLanguageModel
from transformers import TrainingArguments, TrainerCallback
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


# Set Paths and Hyperparameters

In [2]:
# Base path
base_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..', '..'))

# Source and target language
source_language = "Early Modern Bohemian German"
target_language = "English"

# Translation direction
translation_direction = "DE_to_EN" if source_language == "Early Modern Bohemian German" else "EN_to_DE"

# Model parameters
unsloth_model_name = 'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit'
company_name = 'meta'

model_name = unsloth_model_name.split('/')[1]
max_new_tokens = 5000           # Maximum number of model output
max_seq_length = 128000         # Maximum of input tokens
dtype = None                    # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True             # Use 4bit quantization to reduce memory usage. Can be False.

# Model output path
model_output_path = os.path.join(
    base_path, 
    'models', 
    company_name, 
    model_name,
    'finetuning',
    translation_direction)

# Finetuning prompts path
finetuning_prompts_path = os.path.join(
    base_path, 
    'data', 
    'icl_prompts',
    company_name,
    model_name,
    translation_direction, 
    'finetuning_prompt_check.txt'
)

# Print paths
print(f'Company name: {company_name}')
print(f'Model name: {model_name}')
print(f'Base path: {base_path}')
print(f'Translation direction: {translation_direction}')
print(f'Finetuning prompts path: {finetuning_prompts_path}')
print(f'Model output path: {model_output_path}')

# Hugging face login
hub_token = "hf_..."
login(hub_token, add_to_git_credential=True)

Company name: meta
Model name: Meta-Llama-3.1-8B-Instruct-bnb-4bit
Base path: /cs/student/msc/csml/2023/ngriessh/historical_mt
Translation direction: DE_to_EN
Finetuning prompts path: /cs/student/msc/csml/2023/ngriessh/historical_mt/data/icl_prompts/meta/Meta-Llama-3.1-8B-Instruct-bnb-4bit/DE_to_EN/finetuning_prompt_check.txt
Model output path: /cs/student/msc/csml/2023/ngriessh/historical_mt/models/meta/Meta-Llama-3.1-8B-Instruct-bnb-4bit/finetuning/DE_to_EN
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /cs/student/msc/csml/2023/ngriessh/.cache/huggingface/token
Login successful


# Load Training and Validation Dataset

In [3]:
# Data preparation
dataset = load_dataset("niclasgriesshaber/EarlyModernGerman_to_EN_finetuning")
train_dataset = dataset["train"]
validation_dataset = dataset["validation"]

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/3.32M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/370k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

# Prompt template

In [4]:
# Prompt template
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant tasked with translating from {} to {}. NEVER provide an introduction to the translation (e.g. 'Here is the translation:', 'Translate to', 'Hier ist die Übersetzung:', etc.), explanations or clarifications.
NEVER provide a note after your translation.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

### Translation Task. Only translate the following text. Nothing else!

{}:
{}

Translate to {} and match the structure of the source text. Output only this translation and nothing else.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>{}"""

In [5]:
# Apply prompt template
def formatting_prompts_func(examples, source_language, target_language):

    source_texts = examples[source_language]
    target_texts = examples[target_language]
    texts = []

    for source_text, target_text in zip(source_texts, target_texts):
        # Format the prompt with dynamic source and target languages
        text = prompt_template.format(
            source_language, 
            target_language,
            source_language,
            source_text,
            target_language,
            target_text
        ) + '<|eot_id|>'
        texts.append(text)

    return {"text": texts}

# Apply Prompt Template to Validation and Test Dataset

In [6]:
# Apply prompt template to train dataset
train_dataset = train_dataset.map(
    lambda examples: formatting_prompts_func(examples, source_language, target_language),
    batched=True
)

# Apply prompte template to validation dataset
validation_dataset = validation_dataset.map(
    lambda examples: formatting_prompts_func(examples, source_language, target_language),
    batched=True
)

Map:   0%|          | 0/2429 [00:00<?, ? examples/s]

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

In [7]:
# Output a text file to check prompt
with open(finetuning_prompts_path, "w") as f:
    f.write(validation_dataset['text'][0])

# Load Model

In [8]:
# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = unsloth_model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3090 Ti. Max memory: 23.575 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


# PEFT

In [9]:
# PEFT
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Finetuning

In [10]:
# Custom Callback to Save LoRA Parameters and Track Validation Loss after Each Epoch
class SaveLoRAPeftCallback(TrainerCallback):

    def on_epoch_end(self, args, state, control, **kwargs):

        # Get epoch
        num_epoch = round(state.epoch)

        # Print end of epoch
        print(f'End of epoch {num_epoch}')

        # Evaluate the model on the validation dataset
        eval_results = trainer.evaluate()

        # Extract the validation loss
        validation_loss = eval_results["eval_loss"]

        # Print the validation loss
        print(f"Validation Loss after epoch {num_epoch}: {validation_loss}")

        # Save the validation loss to a file
        with open("validation_loss.txt", "a") as f:
            f.write(f"Validation Loss after epoch {num_epoch}: {validation_loss}\n")

        # Create folder for each epoch
        output_dir = os.path.join(args.output_dir, f"epoch_{num_epoch}")
        os.makedirs(output_dir)

        # Print saving LoRA parameters
        print(f"Saving LoRA parameters to {output_dir} at the end of epoch {round(state.epoch)}")
        
        # Save only the LoRA adapters
        kwargs['model'].save_pretrained(output_dir)

        print('Saved. Emptying torch.cuda.empty_cache() now.')
        torch.cuda.empty_cache()

# Initialize the trainer with the callback
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        eval_strategy="epoch",
        save_strategy="no",
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir=model_output_path,
        seed=0,
        log_level="info",
        report_to="none",
    ),
    callbacks=[SaveLoRAPeftCallback()],
)

Map (num_proc=2):   0%|          | 0/2429 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/269 [00:00<?, ? examples/s]

Using auto half precision backend


In [11]:
# Run evaluation at the beginning of the first epoch
print('Evaluation at the beginning of epoch 1')
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 269
  Batch size = 1


Evaluation at the beginning of epoch 1


{'eval_loss': 3.9022915363311768,
 'eval_model_preparation_time': 0.004,
 'eval_runtime': 45.1137,
 'eval_samples_per_second': 5.963,
 'eval_steps_per_second': 5.963}

In [12]:
# Start finetuning
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,429 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 3,035
 "-____-"     Number of trainable parameters = 41,943,040


Epoch,Training Loss,Validation Loss,Model Preparation Time
0,1.238400,1.384910,0.004000
1,1.446900,1.485131,0.004000
2,1.081600,1.328907,0.004000
4,0.249900,1.529425,0.004000



***** Running Evaluation *****
  Num examples = 269
  Batch size = 1


End of epoch 1
Validation Loss after epoch 1: 1.384909987449646
Saving LoRA parameters to /cs/student/msc/csml/2023/ngriessh/historical_mt/models/meta/Meta-Llama-3.1-8B-Instruct-bnb-4bit/finetuning/DE_to_EN/epoch_1 at the end of epoch 1


loading configuration file config.json from cache at /cs/student/msc/csml/2023/ngriessh/.cache/huggingface/hub/models--unsloth--Meta-Llama-3.1-8B-Instruct-bnb-4bit/snapshots/5b0dd3039c312969e7950951486714bff26f0822/config.json
Model config LlamaConfig {
  "_name_or_path": "unsloth/Meta-Llama-3.1-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_

...

Saved. Emptying torch.cuda.empty_cache() now.



***** Running Evaluation *****
  Num examples = 269
  Batch size = 1


End of epoch 5
Validation Loss after epoch 5: 1.5294249057769775
Saving LoRA parameters to /cs/student/msc/csml/2023/ngriessh/historical_mt/models/meta/Meta-Llama-3.1-8B-Instruct-bnb-4bit/finetuning/DE_to_EN/epoch_5 at the end of epoch 5


loading configuration file config.json from cache at /cs/student/msc/csml/2023/ngriessh/.cache/huggingface/hub/models--unsloth--Meta-Llama-3.1-8B-Instruct-bnb-4bit/snapshots/5b0dd3039c312969e7950951486714bff26f0822/config.json
Model config LlamaConfig {
  "_name_or_path": "unsloth/Meta-Llama-3.1-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_

Saved. Emptying torch.cuda.empty_cache() now.
